# Cell构造基础模块

[![下载Notebook](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_notebook.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/master/tutorials/zh_cn/advanced/modules/mindspore_layer.ipynb)&emsp;[![下载样例代码](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_download_code.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/master/tutorials/zh_cn/advanced/modules/mindspore_layer.py)&emsp;[![查看源文件](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/master/tutorials/source_zh_cn/advanced/modules/layer.ipynb)

## Parameter

In [1]:
import numpy as np
import mindspore
from mindspore import nn
from mindspore import ops
from mindspore import Tensor, Parameter

class Network(nn.Cell):
    def __init__(self):
        super().__init__()
        self.w = Parameter(Tensor(np.random.randn(5, 3), mindspore.float32), name='w') # weight
        self.b = Parameter(Tensor(np.random.randn(3,), mindspore.float32), name='b') # bias

    def construct(self, x):
        z = ops.matmul(x, self.w) + self.b
        return z

### Parameter操作

1. 获取Parameter

In [2]:
net = Network()

print(net.trainable_params())

[Parameter(name=w, requires_grad=True, shape=[5, 3], dtype=Float32, value=
[[ 8.06299925e-01, -1.52017653e+00,  1.55664742e-01],
 [ 8.62504691e-02,  1.61433792e+00, -1.72634542e-01],
 [-1.21492767e+00, -2.25986338e+00, -3.15012902e-01],
 [ 1.06195676e+00,  8.28559160e-01, -4.03163850e-01],
 [ 5.97745717e-01, -8.35135400e-01, -4.34642464e-01]]), Parameter(name=b, requires_grad=True, shape=[3], dtype=Float32, value= [ 7.53176391e-01,  5.71677864e-01, -7.82576561e-01])]


In [3]:
print(type(net.parameters_and_names()))

<class 'generator'>


In [4]:
for name, param in net.parameters_and_names():
    print(f"{name}:\n{param.asnumpy()}")

w:
[[ 0.8062999  -1.5201765   0.15566474]
 [ 0.08625047  1.6143379  -0.17263454]
 [-1.2149277  -2.2598634  -0.3150129 ]
 [ 1.0619568   0.82855916 -0.40316385]
 [ 0.5977457  -0.8351354  -0.43464246]]
b:
[ 0.7531764   0.57167786 -0.78257656]


In [5]:
print(net.b)

Parameter(name=b, requires_grad=True, shape=[3], dtype=Float32, value= [ 7.53176391e-01,  5.71677864e-01, -7.82576561e-01])


2. 修改Parameter

In [6]:
net.b.set_data(Tensor([3, 4, 5]))

print(net.b)

Parameter(name=b, requires_grad=True, shape=[3], dtype=Float32, value= [ 3.00000000e+00,  4.00000000e+00,  5.00000000e+00])


In [8]:
from mindspore import jit

@jit
def modify_parameter():
    b_hat = Tensor([7, 8, 9])
    status = ops.assign(net.b, b_hat)
    return status

modify_parameter()
print(net.b)

Parameter(name=b, requires_grad=True, shape=[3], dtype=Float32, value= [ 7.00000000e+00,  8.00000000e+00,  9.00000000e+00])


[ERROR] CORE(28324,7fb716af8740,python):2022-10-17-01:20:18.782.135 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_28324/2294774123.py]


3. Clone Parameter

In [12]:
from mindspore.common.initializer import initializer

x = Parameter(initializer('ones', [1, 2, 3], mindspore.float32))
x_clone = x.clone()
x_clone.name = "x_clone"

print(x)
print(x_clone)

Parameter(name=Parameter, requires_grad=True, shape=[1, 2, 3], dtype=Float32, value=
[[[1., 1., 1.],
  [1., 1., 1.]]])
Parameter(name=x_clone, requires_grad=True, shape=[1, 2, 3], dtype=Float32, value=
[[[ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
  [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00]]])


## Parameter Tuple

变量元组ParameterTuple，用于保存多个Parameter，继承于元组tuple，提供克隆功能。

如下示例提供ParameterTuple创建方法：

In [13]:
from mindspore import ParameterTuple
# 创建
x = Parameter(default_input=ms.Tensor(np.arange(2 * 3).reshape((2, 3))), name="x")
y = Parameter(default_input=initializer('ones', [1, 2, 3], ms.float32), name='y')
z = Parameter(default_input=2.0, name='z')
params = ParameterTuple((x, y, z))

# 从params克隆并修改名称为"params_copy"
params_copy = params.clone("params_copy")

print(params)
print(params_copy)

(Parameter(name=x, requires_grad=True, shape=[2, 3], dtype=Int64, value=
[[0, 1, 2],
 [3, 4, 5]]), Parameter(name=y, requires_grad=True, shape=[1, 2, 3], dtype=Float32, value=
[[[1., 1., 1.],
  [1., 1., 1.]]]), Parameter(name=z, requires_grad=True, shape=[], dtype=Float32, value= 2))
(Parameter(name=params_copy.x, requires_grad=True, shape=[2, 3], dtype=Int64, value=
[[0, 1, 2],
 [3, 4, 5]]), Parameter(name=params_copy.y, requires_grad=True, shape=[1, 2, 3], dtype=Float32, value=
[[[ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
  [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00]]]), Parameter(name=params_copy.z, requires_grad=True, shape=[], dtype=Float32, value= 2))


## 训练状态转换

In [15]:
net.set_train()
print(net.phase)

train


In [16]:
net.set_train(False)
print(net.phase)

predict


## 自定义Cell的反向

In [17]:
class Net(nn.Cell):
    def construct(self, x, y):
        z = x * y
        z = z * y
        return z

    def bprop(self, x, y, out, dout):
        x_dout = x + y
        y_dout = x * y
        return x_dout, y_dout